In [1]:
import numpy as np

In [6]:
def calc_entropy(prob_array):
    if sum(prob_array) != 1:
        return
    return -sum(i * np.log2(i) for i in prob_array)

In [15]:
# calc_entropy([0.17, 0.08, 0.13, 0.62])
# calc_entropy([0.08, 0.12, 0.8])
# calc_entropy([0.17, 0.1, 0.1, 0.63])
# calc_entropy([0.16, 0.03, 0.81])
# calc_entropy([0.02, 0.19, 0.1, 0.69])
# calc_entropy([0.06, 0.3, 0.64])
# calc_entropy([0.23, 0.08, 0.01, 0.68])
calc_entropy([0.02, 0.1, 0.88])

0.6073635558851669